In [1]:
import findspark
findspark.init('/home/srabin/spark-3.5.1-bin-hadoop3')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('cluster_exercise').getOrCreate()

24/08/08 10:06:37 WARN Utils: Your hostname, DESKTOP-P0KTK6U resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/08/08 10:06:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/08 10:06:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/08/08 10:06:38 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/08/08 10:06:38 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [2]:
dataset = spark.read.csv('hack_data.csv', inferSchema= True, header = True)

In [3]:
dataset.head()

Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, Location='Slovenia', WPM_Typing_Speed=72.37)

In [4]:
from pyspark.ml.clustering import KMeans

In [5]:
from pyspark.ml.feature import VectorAssembler

In [7]:
dataset.columns

['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

In [8]:
feat_cols = ['Session_Connection_Time', 'Bytes Transferred', 'Kali_Trace_Used',
 'Servers_Corrupted', 'Pages_Corrupted', 'WPM_Typing_Speed']
# we removed location because the hacker are using vpn and it is useless

In [9]:
assembler = VectorAssembler(inputCols = feat_cols, outputCol = 'features')

In [10]:
final_data = assembler.transform(dataset)

In [11]:
final_data.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)
 |-- features: vector (nullable = true)



In [12]:
from pyspark.ml.feature import StandardScaler

In [13]:
scaler = StandardScaler(inputCol = 'features', outputCol = 'scaledFeatures')

In [14]:
scaler_model = scaler.fit(final_data)
cluster_final_data = scaler_model.transform(final_data)

In [15]:
cluster_final_data.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- scaledFeatures: vector (nullable = true)



## Main tasks: figure out if it was 2 hackers or 3 hackers?

In [16]:
kmeans2 = KMeans(featuresCol = 'scaledFeatures', k = 2)
kmeans3 = KMeans(featuresCol = 'scaledFeatures', k = 3)

In [17]:
model_k2 = kmeans2.fit(cluster_final_data)
model_k3 = kmeans3.fit(cluster_final_data)

24/08/08 10:16:50 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


In [19]:
model_k3.transform(cluster_final_data).select('prediction').show()

+----------+
|prediction|
+----------+
|         0|
|         2|
|         0|
|         0|
|         2|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         2|
|         2|
|         2|
|         0|
|         0|
|         0|
|         2|
|         0|
|         2|
+----------+
only showing top 20 rows



In [20]:
model_k3.transform(cluster_final_data).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         2|   84|
|         0|   83|
+----------+-----+



In [21]:
model_k2.transform(cluster_final_data).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         0|  167|
+----------+-----+

